In [ ]:
# default_exp retrieve

In [ ]:
import config
import requests
import base64
import json
import pandas as pd
import boto3

In [ ]:
#hide
from nbdev.showdoc import *

# retrieval of public tracks

> Pull all tracks and their features for a user's public playlists

In [ ]:
# export
def cred():
    secret_name = "spotify_35"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    client_id = json.loads(get_secret_value_response['SecretString'])['spot_clientID']
    client_secret = json.loads(get_secret_value_response['SecretString'])['spot_clientSECRET']
    access_token = json.loads(get_secret_value_response['SecretString'])['spot_ACC']
    refresh_token = json.loads(get_secret_value_response['SecretString'])['spot_REF']

    return client_id, client_secret, access_token, refresh_token

In [ ]:
c_id, c_secret, access_token, refresh_token = cred()

Get user's public playlists:

In [ ]:
df_items = pd.DataFrame()

user_id = 'atraylor6'
offset = 0

while len(df_items) % 50 == 0:
    
    play_url = f'https://api.spotify.com/v1/users/{user_id}/playlists?limit=50&offset={offset}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    r_play = requests.get(play_url, headers=headers)
    

    if r_play.status_code == 401:

        TOKEN_URL = 'https://accounts.spotify.com/api/token'

        message = config.CLIENT_ID + ':' + config.CLIENT_SECRET
        messageBytes = message.encode('ascii')
        base64Bytes = base64.b64encode(messageBytes)
        base64Message = base64Bytes.decode('ascii')
    
        headers = {
        'Authorization': 'Basic ' + base64Message,
        'Content-Type': 'application/x-www-form-urlencoded'
        }

        pars_refresh = {
            'grant_type': 'refresh_token',
            'refresh_token': refresh_token,
            'redirect_uri': 'http://localhost:8888/callback',
        }

        r_refresh = requests.post(TOKEN_URL, headers=headers, params=pars_refresh)
        access_token = r_refresh.json()['access_token']

        headers = {
        'Authorization': f'Bearer {access_token}'
                    }

        r_play = requests.get(play_url, headers=headers)
        
    r_play = r_play.json()
    # print(r_play['items'][0])
    df_items = df_items.append(r_play['items'], ignore_index=True)
    # print(df_items.head())
    offset += 50

df_pub = df_items[df_items['public'] == True]

Get track and artist names/ids for each track in public playlists:

In [ ]:
df_list = []

for p_id in df_pub['id']:

    df_tracks = pd.DataFrame()

    offset = 0

    while len(df_tracks) % 100 == 0:
        
        track_url = f'https://api.spotify.com/v1/playlists/{p_id}/tracks?limit=100&offset={offset}'
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        r_track = requests.get(track_url, headers=headers)

        track_ids = [t['track']['id'] for t in r_track.json()['items']]
        track_names = [t['track']['name'] for t in r_track.json()['items']]
        track_artists = [t['track']['artists'][0]['name'] for t in r_track.json()['items']]
        artist_id = [t['track']['artists'][0]['id'] for t in r_track.json()['items']]

        df_t = pd.DataFrame({
            'id': track_ids,
            'name': track_names,
            'artist': track_artists,
            'artist id': artist_id,
            'playlist id': p_id

        })  

        join_ids = ','.join(track_ids)
        feat_url = f'https://api.spotify.com/v1/audio-features?limit=100&offset={offset}&ids={join_ids}'
        r_feat = requests.get(feat_url, headers=headers)
        feat_frame = pd.DataFrame(r_feat.json()['audio_features'])

        df_t = pd.merge(df_t, feat_frame, on='id')

        df_tracks = df_tracks.append(df_t)
        offset += 100

    df_list.append(df_tracks)

df_tracks = pd.concat(df_list)
df_tracks = df_tracks.drop_duplicates()
id_map = df_pub[['id', 'name']].set_index('id').to_dict()['name']
df_tracks['playlist id'] = df_tracks['playlist id'].map(id_map)
df_tracks = df_tracks.reset_index(drop=True)
print(df_tracks.shape)

(1145, 22)


In [ ]:
artist_list = df_tracks['artist id'].tolist()
genre_list = []
index = 0
temp_list = artist_list[index: index+50]

for i in range(round(len(artist_list)/50)):
    artist_join = ','.join(temp_list)
    art_url = f'https://api.spotify.com/v1/artists?ids={artist_join}'
    r_art = requests.get(art_url, headers=headers)
    
    try:
        g = [i['genres'] for i in r_art.json()['artists']]
    except IndexError:
        g = ['No Genre']

    genre_list.extend(g)
    index += 50
    temp_list = artist_list[index: index+50]


In [ ]:
g_ser = pd.Series(genre_list)
# g_ser.apply(lambda x: len(x)).value_counts()

In [ ]:
trimmed_g = g_ser.apply(lambda x: x[:3])
# trimmed_g.apply(lambda x: len(x)).value_counts()

In [ ]:
df_tracks['genre'] = trimmed_g
# df_tracks.tail()

,id,name,artist,artist id,playlist id,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,genre
1140,17Xxp5dtS4GmdGjNZNb5Gk,Track07,Denzel Curry,6fxyWrfmjcbj5d12gXeiNV,Flowgurt,0.609,0.623,10,-7.865,0,...,0.4040,0.369,93.929,audio_features,spotify:track:17Xxp5dtS4GmdGjNZNb5Gk,https://api.spotify.com/v1/tracks/17Xxp5dtS4Gm...,https://api.spotify.com/v1/audio-analysis/17Xx...,123520,4,"[dark trap, drill, hip hop]"
1141,6YasuOZcSrwQvHNbsxuPPW,Truffles,Mick Jenkins,1FvjvACFvko2Z91IvDljrx,Flowgurt,0.841,0.649,11,-4.151,1,...,0.1310,0.615,132.999,audio_features,spotify:track:6YasuOZcSrwQvHNbsxuPPW,https://api.spotify.com/v1/tracks/6YasuOZcSrwQ...,https://api.spotify.com/v1/audio-analysis/6Yas...,195000,4,"[chicago rap, conscious hip hop, drill]"
1142,4CGORuRtjSK6xMy8s1VvSS,The Mission,Bakar,3K2Srho6NCF3o9MswGR76H,Flowgurt,0.708,0.437,10,-6.982,0,...,0.0976,0.393,94.053,audio_features,spotify:track:4CGORuRtjSK6xMy8s1VvSS,https://api.spotify.com/v1/tracks/4CGORuRtjSK6...,https://api.spotify.com/v1/audio-analysis/4CGO...,191479,4,[uk alternative hip hop]
1143,171PWhxgdrIOgcAiBPO3Gr,The Count (feat. Wiz Khalifa),Curren$y,6X8WdFjrNhXATMDSs26aCc,Flowgurt,0.385,0.757,0,-5.960,1,...,0.2350,0.469,82.673,audio_features,spotify:track:171PWhxgdrIOgcAiBPO3Gr,https://api.spotify.com/v1/tracks/171PWhxgdrIO...,https://api.spotify.com/v1/audio-analysis/171P...,194595,4,"[alternative hip hop, dirty south rap, gangste..."
1144,4WqbpRNAcjRq256ExCTR59,94 Camry Music,Femdot,7aGhkUVp7V3klWfTFe7AHS,Flowgurt,0.688,0.641,1,-6.433,1,...,0.1300,0.549,87.374,audio_features,spotify:track:4WqbpRNAcjRq256ExCTR59,https://api.spotify.com/v1/tracks/4WqbpRNAcjRq...,https://api.spotify.com/v1/audio-analysis/4Wqb...,221000,4,"[indie hip hop, underground hip hop]"


In [ ]:
genre_bin = pd.get_dummies(df_tracks['genre'].explode())
genre_bin = genre_bin.groupby(level=0).sum()

In [ ]:
df_tracks = df_tracks.drop('genre', axis=1)

In [ ]:
df_tracks = pd.concat([df_tracks, genre_bin], axis=1)

In [ ]:
df_tracks.to_csv('features_spot.csv')